In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
%reload_ext autoreload

In [3]:
from IPython.display import clear_output

# !pip install torchvision==0.11.1 
# !pip install torchsummary 
# !pip install torchmetrics 
# !pip install pyDOE
# !pip install git+https://github.com/Fangyh09/pytorch-receptive-field.git 
    
clear_output()

In [4]:
from IPython.display import clear_output
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torch_receptive_field import receptive_field
from skimage import feature, filters, morphology
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import copy
import importlib
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

import data_preprocessing
from data_preprocessing import ParLinnaeus5, ParLinnaeus5_v2, Linnaeus5, get_data, get_dataloaders
from models import ConvNet
from early_stopping import EarlyStopping
import experiment
from experiment import run_experiment
from metrics import define_comparison_table
from utils import load_config, load_trained_model
from visualization import imshow, show_PIL

config = load_config("config.yaml")
config

{'DATA_DIR': '/home/jovyan/Linnaeus 5/Linnaeus 5 128X128/',
 'MODELS_DIR': '/home/jovyan/Models/',
 'IMG_SIZE': 128,
 'GAMMA': 2.2,
 'BATCH_SIZE': 8,
 'EPOCHS': 75,
 'NUM_SET_PARAMS': 1000,
 'SIGMA': 1,
 'RADIUS': 20,
 'WIDTH': 40,
 'K': 0.8,
 'WIN_SIZE': 25}

### 1] Матрица коэффициентов линейной комбинации фильтров одна для всех слоев

1) по 64 фильтра на слое

In [7]:
BS = 10
IN = 32
NUM = 12

In [17]:
param = ParamBlock(3, NUM)

In [18]:
params_values = torch.randn([BS, 3])
param = ParamBlock(3, NUM)
coeffs = param(params_values)
coeffs.shape

torch.Size([10, 1, 12])

In [19]:
conv = nn.Conv2d(in_channels=IN*NUM, out_channels=IN*NUM, kernel_size=3, padding=1, groups=NUM)
img = torch.randn([BS, IN, 128, 128])
x = img.repeat(1, NUM, 1, 1)
torch.cat(conv(x).chunk(NUM, dim=1), dim=0).view(-1, NUM, IN, 128, 128).shape

torch.Size([10, 12, 32, 128, 128])

In [20]:
kernels = torch.cat(conv(x).chunk(NUM, dim=1), dim=0).view(-1, NUM, IN*128*128)
kernels.shape

torch.Size([10, 12, 524288])

In [21]:
# kernels = torch.cat(conv(x).chunk(NUM, dim=1), dim=0).view(-1, NUM, 16, 128, 128).view(-1, 16*128*128, NUM)
torch.bmm(coeffs, kernels).view(-1, IN, 128, 128).shape

torch.Size([10, 32, 128, 128])

In [29]:
class ParamBlock(nn.Module):
    def __init__(self, num_features, num_coeffs, bias=True):
        '''
        num_features: number of parameters of an approximated filter
        in_channels:  number of input channels (equal to number of filters in the previous hidden layer)
        out_channels: number of output channels (equal to number of new filters 
                      that are linear combinations of the filters from previous hidden layer)
        '''
        super(ParamBlock, self).__init__()
        
        self.param_block = nn.Sequential(
            nn.Linear(in_features=num_features, out_features=5*num_coeffs, bias=bias),
            nn.LeakyReLU(0.02),
            nn.Linear(in_features=5*num_coeffs, out_features=10*num_coeffs, bias=bias),
            nn.LeakyReLU(0.02),
            nn.Linear(in_features=10*num_coeffs, out_features=num_coeffs, bias=bias)
        )

    def forward(self, x):
        out = self.param_block(x)
        out = F.tanh(out)
        return out.unsqueeze(1)
    
###### для каждого слоя своя матрица интерполяции

class ParamConv(nn.Conv2d):
    def __init__(self, in_channels, img_size, num_features, num_coeffs):       
        super(ParamConv, self).__init__(in_channels, in_channels, kernel_size=3, bias=False)
        self.in_channels = in_channels
        self.img_size = img_size
        self.num_coeffs = num_coeffs
        
        self.conv = nn.Conv2d(in_channels=in_channels*num_coeffs,
                              out_channels=in_channels*num_coeffs,
                              kernel_size=3, padding=1, groups=self.num_coeffs)
        
        self.param_block = ParamBlock(num_features, num_coeffs)
        
        
    def forward(self, img, parameters):
        # stack the input image <num_coeffs> times
        img = img.repeat(1, self.num_coeffs, 1, 1)
#         print(img.shape)
        
        #flatten kernels
        kernels = torch.cat(self.conv(img).chunk(self.num_coeffs, dim=1), dim=0)
#         kernels = kernels.view(-1, self.in_channels*self.img_size*self.img_size, self.num_coeffs)
        kernels = kernels.view(-1, self.num_coeffs, self.in_channels*self.img_size*self.img_size)

        # coeffs
        coeffs = self.param_block(parameters)

#         out = torch.bmm(kernels, coeffs).view(-1, self.in_channels, self.img_size, self.img_size)
        out = torch.bmm(coeffs, kernels).view(-1, self.in_channels, self.img_size, self.img_size)
        out = out.view(-1, self.in_channels, self.img_size, self.img_size)
        return out
 

######

class ParamNet(nn.Module):
    def __init__(self, filter_name, num_features, in_channels, mid_channels, num_coeffs, img_size=128, threshold=0.5):
        super().__init__()
        self.filter_name = filter_name
        self.threshold = threshold
        
        self.conv0  = nn.Conv2d(in_channels=in_channels,
                                out_channels=mid_channels,
                                kernel_size=3, padding=1)
        
        self.conv1  = ParamConv(in_channels=mid_channels,
                                img_size=img_size,
                                num_features=num_features,
                                num_coeffs=num_coeffs)
        
        self.conv2  = ParamConv(in_channels=mid_channels,
                                img_size=img_size,
                                num_features=num_features,
                                num_coeffs=num_coeffs)
        
        self.conv3  = ParamConv(in_channels=mid_channels,
                                img_size=img_size,
                                num_features=num_features,
                                num_coeffs=num_coeffs)
        
        self.conv10 = ParamConv(in_channels=mid_channels,
                                img_size=img_size,
                                num_features=num_features,
                                num_coeffs=num_coeffs)
        
        self.conv11 = nn.Conv2d(in_channels=mid_channels,
                                out_channels=1,
                                kernel_size=3, padding=1) 
        
    def forward(self, x, params):        
      x = F.leaky_relu_(self.conv0(x), 0.02)                        
      x = F.leaky_relu_(self.conv1(x, params), 0.02)
      x = F.leaky_relu_(self.conv2(x, params), 0.02)
      x = F.leaky_relu_(self.conv3(x, params), 0.02)
      x = F.leaky_relu_(self.conv10(x, params), 0.02)
      x = self.conv11(x)
 
      probas = torch.sigmoid(x)
      if self.filter_name in ['canny', 'niblack']:
        y_hat = (probas>=self.threshold).float()
      else:
        y_hat = probas
      return probas, y_hat

In [26]:
paramnet = ParamNet(filter_name='canny',
                   num_features=1,
                   in_channels=1,
                   mid_channels=12,
                   num_coeffs=12)

img = torch.randn([10, 1, 128, 128])
params = torch.randn([10, 1])
# paramnet(img, params)[0]

In [30]:
# параметрическая сеть без добавления параметров в каналы входного изображения
paramnet = ParamNet(filter_name='canny',
                   num_features=3,
                   in_channels=1,
                   mid_channels=12,
                   num_coeffs=12)

optimizer = torch.optim.Adam(paramnet.parameters())
lr_sched = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=8)
loss = nn.BCELoss()
early_stopping = EarlyStopping(patience=11)

paramnet, history, metrics_calc, writer11 = run_experiment(filter_name='canny', 
                                                model=paramnet,
                                                optimizer=optimizer, 
                                                criterion=loss,
                                                dataset=ParLinnaeus5_v2,
                                                is_parameterized=True,
                                                is_linear=False,
                                                use_cache=False,
                                                scheduler=lr_sched,
                                                stopping=early_stopping, 
                                                model_name='paramnet2-canny',
                                                add_channel=False,
                                                comp_table=None,
                                                writer=None,
                                                start_idx=0)

  0%|          | 0/750 [00:00<?, ?it/s]

Files are loaded.
Train size:  6000 	 Valid size:  1000 	 Test size:  1000

************    Starting experiment    ************

Model: ParamNet(
  (conv0): Conv2d(1, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1): ParamConv(
    12, 12, kernel_size=(3, 3), stride=(1, 1), bias=False
    (conv): Conv2d(144, 144, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=12)
    (param_block): ParamBlock(
      (param_block): Sequential(
        (0): Linear(in_features=3, out_features=60, bias=True)
        (1): LeakyReLU(negative_slope=0.02)
        (2): Linear(in_features=60, out_features=120, bias=True)
        (3): LeakyReLU(negative_slope=0.02)
        (4): Linear(in_features=120, out_features=12, bias=True)
      )
    )
  )
  (conv2): ParamConv(
    12, 12, kernel_size=(3, 3), stride=(1, 1), bias=False
    (conv): Conv2d(144, 144, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=12)
    (param_block): ParamBlock(
      (param_block): Sequential(
        (

  0%|          | 2/750 [00:00<01:02, 12.03it/s]

Metrics cannot be calculated
Epoch 1:	 mean train_loss: 0.376951	 mean val_loss 0.359781



  0%|          | 2/750 [00:00<01:09, 10.73it/s]

...

  0%|          | 2/750 [00:00<01:04, 11.66it/s]

Metrics cannot be calculated
Epoch 6:	 mean train_loss: 0.280699	 mean val_loss 0.273794

INFO: Early stopping counter 1 of 11


100%|██████████| 750/750 [01:01<00:00, 12.23it/s]


Metrics cannot be calculated
Epoch 7:	 mean train_loss: 0.275867	 mean val_loss 0.270180

The best loss on validation is reached. Saving the model.
Model is not a DataParallel object.

Model's weights were saved. Name:  paramnet2-canny-7 

Params: 0.17538455412447 0.13623946175762658 0.17271402392745977
Params: 0.5995657468036464 0.17085813269766548 0.3186137393438979
Params: 2.9935683721175685 0.12363669989469714 0.2456785969003133
Params: 2.42851868485842 0.17980780526954004 0.22450655877756123
Params: 0.5925045623296372 0.15755130101023176 0.2243691658966225
Params: 1.2775920502773959 0.19533340269841618 0.38877697000866396
Params: 2.3030600716577743 0.17745641763516373 0.22715122241713365
Params: 1.8132921695453434 0.1207803564216556 0.14013041683230051


AttributeError: 'list' object has no attribute 'cuda'